In [26]:
import os, re

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

os.environ["HF_HOME"] = r"C:\Users\241011\Documents\models"
# NLLB-200 모델과 토크나이저 로드
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [60]:
def split_text(text, max_length=50):
    paragraph = re.split(r'(?<=\.)\s*', text)
    print(paragraph)
    chunks = []
    current_chunk = ""
    
    for sentence in paragraph:
        # 현재 문장이 추가될 경우 최대 길이를 초과하면, 현재까지 문장을 chunk로 저장
        if len(current_chunk) + len(sentence) <= max_length:
            current_chunk += sentence + " "  # 문장 추가 + 뒤에 공백 추가
        else: # 크기가 초과 될 경우 
            if current_chunk:  # 기존에 있는 chunk가 있으면 저장(처음에 통과 방지)
                chunks.append(current_chunk.strip()) # 앞 뒤 공백 제거
            current_chunk = sentence + " "  # 새로운 chunk 시작

    if current_chunk: # 마지막 찌꺼기 문장이 100자 넘지 않을 경우 
        chunks.append(current_chunk.strip())
    return chunks

# text = "This is a sample text. It contains several sentences. We want to split it into smaller chunks. However, we must split it at periods, not in the middle of sentences."

In [62]:
# 번역할 문장
sentence = input()

chunks = split_text(text, max_length=100)
print(chunks)

for chunk in chunks: # 인덱스 값과 같이 출력 함수
    inputs = tokenizer(chunk, return_tensors='pt') # NLLB-200에서 영어(Latin)와 한국어(Hangul) 코드 설정
    generated_tokens = model.generate(inputs.input_ids, forced_bos_token_id = tokenizer.convert_tokens_to_ids('kor_Hang'), max_length=200) # 입력 문장에 대한 번역 수행 (영어 -> 한국어)
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True) # 번역 결과 디코딩
    print(f'Translated text: {translated_text}')

 This is a sample text. It contains several sentences. We want to split it into smaller chunks. However, we must split it at periods, not in the middle of sentences


['This is a sample text.', 'It contains several sentences.', 'We want to split it into smaller chunks.', 'However, we must split it at periods, not in the middle of sentences.', '']
['This is a sample text. It contains several sentences. We want to split it into smaller chunks.', 'However, we must split it at periods, not in the middle of sentences.']
Translated text: 이것은 샘플 텍스트입니다. 몇 개의 문장을 포함합니다. 우리는 그것을 작은 조각으로 나누고 싶습니다.
Translated text: 하지만 우리는 문장을 중간에서 나누는 것이 아니라 기간으로 나누어야 합니다.


In [37]:
text = '안녕하세요. 반값습니다.'

sentences = text.split(".")
sentences

['안녕하세요', ' 반값습니다', '']